***Check GPU***

In [ ]:
!pip install Theano==1.0

In [ ]:
from theano import function, config, shared, tensor
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tensor.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, tensor.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

***Install Google Drive Libraries and Authorization***

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

***Mount Google Drive***

Drive folder is created under content/

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
!ls

***Install Neupy***

In [ ]:
!pip install neupy
!pip install tqdm

## Function

In [ ]:
# Plot history for accuracy and loss
def plot_model(model_details, train_acc_score, val_acc_score):
    # Create sub-plots
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    
    # Summarize history for accuracy
    axs[0].plot(range(1,len(train_acc_score)+1),train_acc_score)
    axs[0].plot(range(1,len(val_acc_score)+1),val_acc_score)
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(train_acc_score)+1),len(train_acc_score)/10)
    axs[0].legend(['train', 'val'], loc='best')
    
    # Summarize history for loss
    axs[1].plot(range(1,len(model_details.train_errors)+1),model_details.train_errors)
    axs[1].plot(range(1,len(model_details.validation_errors)+1),model_details.validation_errors)
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_details.train_errors)+1),len(model_details.train_errors)/10)
    axs[1].legend(['train', 'val'], loc='best')

# Plot history for accuracy and loss
def plot_loadmodel(train_loss, val_loss, train_acc_score, val_acc_score):
    # Create sub-plots
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    
    # Summarize history for accuracy
    axs[0].plot(range(1,len(train_acc_score)+1),train_acc_score)
    axs[0].plot(range(1,len(val_acc_score)+1),val_acc_score)
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(train_acc_score)+1),len(train_acc_score)/10)
    axs[0].legend(['train', 'val'], loc='best')
    
    # Summarize history for loss
    axs[1].plot(range(1,len(train_loss)+1),train_loss)
    axs[1].plot(range(1,len(val_loss)+1),val_loss)
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(train_loss)+1),len(train_loss)/10)
    axs[1].legend(['train', 'val'], loc='best')
        
    # Show the plot
    plt.show()
    
# Plot confusion matrix of prediction value
def plot_confusion_matrix(cm, classes):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Normalized confusion matrix")

    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, '',
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def visualize_errors(images_test, labels_test, class_names, labels_pred, correct):
    incorrect = (correct == False)
    
    # Images of the test-set that have been incorrectly classified.
    images_error = images_test[incorrect]
    print(images_error.shape)
    
    # Get predicted classes for those images
    labels_error = labels_pred[incorrect]
    
    # Print the unique misclassified classes
    print(np.unique(labels_error))
    print([id_to_label[i] for i in np.unique(labels_error)])

    # Get true classes for those images
    labels_true = labels_test[incorrect]
    
    
    # Plot the first 9 images.
    plot_images(images=images_error[0:9],
                labels_true=labels_true[0:9],
                class_names=class_names,
                labels_pred=labels_error[0:9])

    return images_error, labels_true

# Plot error prediction
def plot_images(images, labels_true, class_names, labels_pred=None):

    assert len(images) == len(labels_true)

    # Create a figure with sub-plots
    fig, axes = plt.subplots(3, 3, figsize = (8,8))

    # Adjust the vertical spacing
    if labels_pred is None:
        hspace = 0.2
    else:
        hspace = 0.5
    fig.subplots_adjust(hspace=hspace, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Fix crash when less than 9 images
        if i < len(images):
            # Plot the image
            ax.imshow(images[i], interpolation='spline16')
            
            # Name of the true class
            labels_true_name = class_names[labels_true[i]]

            # Show true and predicted classes
            if labels_pred is None:
                xlabel = "True: "+labels_true_name
            else:
                # Name of the predicted class
                labels_pred_name = class_names[labels_pred[i]]

                xlabel = "True: "+labels_true_name+"\nPredicted: "+ labels_pred_name

            # Show the class on the x-axis
            ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot
        ax.set_xticks([])
        ax.set_yticks([])
    
    # Show the plot
    plt.show()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import glob
import os
import dill
import itertools # confusion matrix
from tqdm import tqdm

import theano
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn import metrics
from neupy import algorithms, layers, environment, plots

Change directory to your preferred folder

***Copy data to local vm content folder***

In [ ]:
os.chdir("../../../") #change directory to content folder

!git clone https://github.com/Horea94/Fruit-Images-Dataset.git

In [ ]:
!ls -a Fruit-Images-Dataset

In [ ]:
!pwd

In [ ]:
!ls

**Preparing Training Data**

Check if data is present in local vm

In [ ]:
glob.glob("../../../Fruit-Images-Dataset/Training/*")

In [ ]:
glob.glob("C:/Users/Shi Yuan/Google Drive/Colab/fruits_classification/Training/*")

In [ ]:
all_train_fruit_img = []
all_train_label = []
for dir_path in tqdm(glob.glob("C:/Users/Shi Yuan/Google Drive/Colab/fruits_classification/Training/*")):
    img_label = dir_path.split("\\")[-1]
    for image_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (32, 32))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        all_train_fruit_img.append(image)
        all_train_label.append(img_label)

all_train_fruit_img = np.array(all_train_fruit_img)
all_train_label = np.array(all_train_label)

In [ ]:
class_label = np.unique(all_train_label)
class_label

In [ ]:
label_to_id = {v:k for k,v in enumerate(class_label) }
id_to_label = {v:k for k,v in label_to_id.items() }

id_to_label

In [ ]:
all_train_label_id = np.array([label_to_id[i] for i in all_train_label])
all_train_label_id

In [ ]:
all_train_fruit_img.shape, all_train_label_id.shape, class_label.shape

In [ ]:
y = np.bincount(all_train_label_id)
ii = np.nonzero(y)[0]
plt.bar(np.unique(all_train_label_id), y[ii], color="#87CEFA", edgecolor="Black")
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Historgram of each class label in all_train image set')

## Splitting into train and validation sets (hold-out validation)

In [ ]:
# Splitting, with stratify class
train_fruit_img, validate_fruit_img, train_label_id, validate_label_id = train_test_split(all_train_fruit_img, all_train_label_id, test_size=0.2, random_state=None, stratify=all_train_label_id)

In [ ]:
train_fruit_img.shape, train_label_id.shape, validate_fruit_img.shape, validate_label_id.shape

In [ ]:
# Plot histogram of train image set
y = np.bincount(train_label_id)
ii = np.nonzero(y)[0]
plt.subplot(2, 1, 1)
plt.tight_layout()
plt.bar(np.unique(train_label_id), y[ii], color="#87CEFA", edgecolor="Black")
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Historgram of each class label in train image set')

# Plot histogram of validate image set
y = np.bincount(validate_label_id)
ii = np.nonzero(y)[0]
plt.subplot(2, 1, 2)
plt.tight_layout()
plt.bar(np.unique(validate_label_id), y[ii], color="#87CEFA", edgecolor="Black")
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Historgram of each class label in validate image set')

In [ ]:
validate_label_id # already in id

Save the split train and validate data

In [ ]:
np.save('train_fruit_img.npy', train_fruit_img)
np.save('train_label_id.npy', train_label_id)

np.save('validate_fruit_img.npy', validate_fruit_img)
np.save('validate_label_id.npy', validate_label_id)

**Preparing Test Data**

In [ ]:
test_fruit_img=[]
test_label =[]
for dir_path in tqdm(glob.glob("C:/Users/Shi Yuan/Google Drive/Colab/fruits_classification/Validation/*")):
    img_label = dir_path.split("\\")[-1]
    for image_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (32, 32))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        test_fruit_img.append(image)
        test_label.append(img_label)

test_fruit_img = np.array(test_fruit_img)
test_label = np.array(test_label)

In [ ]:
test_label_id = np.array([label_to_id[i] for i in test_label])

In [ ]:
test_fruit_img.shape, test_label_id.shape

In [ ]:
np.save('test_fruit_img.npy', test_fruit_img)
np.save('test_label_id.npy', test_label_id)

In [ ]:
# Plot histogram of test image set
y = np.bincount(test_label_id)
ii = np.nonzero(y)[0]
plt.subplot(2, 1, 2)
plt.tight_layout()
plt.bar(np.unique(test_label_id), y[ii], color="#87CEFA", edgecolor="Black")
plt.xlabel('Class Labels')
plt.ylabel('Count')
plt.title('Historgram of each class label in test image set')

**Defining Train and Test Images**

In [ ]:
# Show first 10 images
print(train_label_id[0:10])

slice = 10
plt.figure(figsize=(16,8))
for i in range(slice):
    plt.subplot(1, slice, i+1)
    #plt.tight_layout()
    plt.imshow(train_fruit_img[i], interpolation='nearest')
    plt.axis('off')
    plt.title('{}'.format(id_to_label[train_label_id[i]]), fontsize=8)

print(validate_label_id[0:10])

slice = 10
plt.figure(figsize=(16,8))
for i in range(slice):
    plt.subplot(1, slice, i+1)
    plt.imshow(validate_fruit_img[i], interpolation='nearest')
    plt.axis('off')
    plt.title('{}'.format(id_to_label[validate_label_id[i]]), fontsize=8)

print(test_label_id[0:10])

slice = 10
plt.figure(figsize=(16,8))
for i in range(slice):
    plt.subplot(1, slice, i+1)
    plt.imshow(test_fruit_img[i], interpolation='nearest')
    plt.axis('off')
    plt.title('{}'.format(id_to_label[test_label_id[i]]), fontsize=8)

In [ ]:
# convert this shape (n_samples, height, width, n_channels)
# to (n_samples, n_channels, height, width)
x_train = np.transpose(train_fruit_img, (0, 3, 1, 2))
x_train = x_train.astype(np.float32, copy=False)

x_validate = np.transpose(validate_fruit_img, (0, 3, 1, 2))
x_validate = x_validate.astype(np.float32, copy=False)

x_test = np.transpose(test_fruit_img, (0, 3, 1, 2))
x_test = x_test.astype(np.float32, copy=False)

mean = x_train.mean(axis=(0, 2, 3)).reshape((1, -1, 1, 1))
std = x_train.std(axis=(0, 2, 3)).reshape((1, -1, 1, 1))

x_train -= mean
x_train /= std
x_validate -= mean
x_validate /= std
x_test -= mean
x_test /= std

target_scaler = OneHotEncoder()
y_train = target_scaler.fit_transform(train_label_id.reshape((-1, 1))).todense()
y_validate = target_scaler.fit_transform(validate_label_id.reshape((-1, 1))).todense()
y_test = target_scaler.transform(test_label_id.reshape((-1, 1))).todense()


In [ ]:
x_train.shape, y_train.shape, x_validate.shape, y_validate.shape, x_test.shape, y_test.shape

Save train dataset mean and std to reproduce in future

In [ ]:
np.save('meanfile.npy', mean)
np.save('stdfile.npy', std)

## Add accuracy prediction after every epoch

In [ ]:
def on_epoch_end(network):
    train_predicted = network.predict(x_train).argmax(axis=1)
    y_train_labels = np.asarray(y_train.argmax(axis=1)).reshape(len(y_train))  
    
    validate_predicted = network.predict(x_validate).argmax(axis=1)
    y_validate_labels = np.asarray(y_validate.argmax(axis=1)).reshape(len(y_validate))  
    
    test_predicted = network.predict(x_test).argmax(axis=1)
    y_test_labels = np.asarray(y_test.argmax(axis=1)).reshape(len(y_test))

    
    train_acc_score.append(metrics.accuracy_score(y_train_labels, train_predicted))
    val_acc_score.append(metrics.accuracy_score(y_validate_labels, validate_predicted))
    test_acc_score.append(metrics.accuracy_score(y_test_labels, test_predicted))
    
    print('Train Accuracy Score: {}'.format(train_acc_score))
    print('Validation Accuracy Score: {}'.format(val_acc_score))
    print('Test Accuracy Score: {}'.format(test_acc_score))
    
    current_epoch = network.last_epoch
    
    filedir = "decay{}_step{}_batchsize{}/network-storage.dill".format(param['decay'], param['step'], param['batch_size'])
    
    if (current_epoch > 1):
        if (network.validation_errors[current_epoch-1] < network.validation_errors[best_epoch[-1]-1]):
            # save the best model
            with open(filedir, 'wb') as f:
                dill.dump(network, f)
            best_epoch.append(current_epoch)
    else :
        with open(filedir, 'wb') as f:
            dill.dump(network, f)
        best_epoch.append(current_epoch)
        
    print('Best Epoch: {}'.format(best_epoch[-1]))

## Initialise Network

In [ ]:
def save_network_score(filedir):
    # Save training and validation curves
    np.save(filedir+'train_acc_score.npy', train_acc_score)
    np.save(filedir+'val_acc_score.npy', val_acc_score)
    np.save(filedir+'test_acc_score.npy', test_acc_score)

    np.save(filedir+'train_loss.npy', network.train_errors)
    np.save(filedir+'val_loss.npy', network.validation_errors)

    np.save(filedir+'best_epoch.npy', best_epoch)

In [ ]:
from sklearn.model_selection import ParameterGrid

param_grid = {'step': [0.4, 0.5], 'decay': [1], 'batch_size': [256]}

param_list = list(ParameterGrid(param_grid))
param_list

In [ ]:
len(param_list)

In [ ]:
results_df =  pd.DataFrame()
for param in param_list:
    print('Decay: {}, Learning Rate: {}, Batch Size: {}'.format(param['decay'], param['step'], param['batch_size']))
    filedir = "decay{}_step{}_batchsize{}/".format(param['decay'], param['step'], param['batch_size'])
    
    if not os.path.exists(filedir):
        os.makedirs(filedir)
    
    train_acc_score = []
    val_acc_score = []
    test_acc_score = []
    best_epoch = []

    network = algorithms.Adadelta(
        [
            layers.Input((3, 32, 32)),

            layers.Convolution((32, 3, 3)) > layers.BatchNorm() > layers.PRelu(),
            
            layers.Convolution((32, 3, 3)) > layers.BatchNorm() > layers.PRelu(),
            layers.MaxPooling((2, 2)),
            
            layers.Convolution((64, 3, 3)) > layers.BatchNorm() > layers.PRelu(),

            layers.Convolution((64, 3, 3)) > layers.BatchNorm() > layers.PRelu(),
            layers.MaxPooling((2, 2)),
            
            layers.Convolution((128, 3, 3)) > layers.BatchNorm() > layers.PRelu(),
            layers.MaxPooling((2, 2)),

            layers.Reshape(),

            layers.Linear(1024) > layers.BatchNorm() > layers.PRelu(),
            layers.Softmax(60),
        ],

        error='categorical_crossentropy',
        step=param['step'],
        decay=param['decay'],
        shuffle_data=True,
        batch_size=param['batch_size'], #128
        verbose=True,
        epoch_end_signal=on_epoch_end,
    )
 
    
    network.train(x_train, y_train, x_validate, y_validate, epochs=50)
    save_network_score(filedir)
    plot_model(network, train_acc_score, val_acc_score)
    plt.savefig(filedir+'plot.png', bbox_inches='tight')
    
    df = pd.DataFrame(data=param, index=[0])
    d = {'min validation loss': min(network.validation_errors), 'best epoch': best_epoch[-1], 
         'train loss': network.train_errors[best_epoch[-1]-1], 'train_acc_score': train_acc_score[best_epoch[-1]-1], 
         'val_acc_score': val_acc_score[best_epoch[-1]-1], 'test_acc_score': test_acc_score[best_epoch[-1]-1],}
    df2 = pd.DataFrame(data=d, index=[0])
    concat_df = pd.concat([df, df2], axis=1)

    results_df = results_df.append(concat_df)

timestr = time.strftime("%Y%m%d-%H%M%S")
results_df.to_csv('results_'+timestr+'.csv', encoding='utf-8', index=False)
results_df

In [ ]:
network.architecture()

**Load The Best Trained Network**

In [ ]:
import dill

with open('Architecture 15/decay1_step0.4_batchsize256/network-storage.dill', 'rb') as f:
    network = dill.load(f)

Load the mean and std

In [ ]:
validate_predicted = network.predict(x_validate).argmax(axis=1)
y_validate_labels = np.asarray(y_validate.argmax(axis=1)).reshape(len(y_validate))  

print(metrics.classification_report(y_validate_labels, validate_predicted))
score = metrics.accuracy_score(y_validate_labels, validate_predicted)
print("Validate Accuracy: {:.2%}".format(score))
print(metrics.confusion_matrix(validate_predicted, y_validate_labels))

In [ ]:
test_predicted = network.predict(x_test).argmax(axis=1)
y_test_labels = np.asarray(y_test.argmax(axis=1)).reshape(len(y_test))

print(metrics.classification_report(y_test_labels, test_predicted))
score = metrics.accuracy_score(y_test_labels, test_predicted)
print("Test Accuracy: {:.2%}".format(score))
print(metrics.confusion_matrix(test_predicted, y_test_labels))

correct = (test_predicted == y_test_labels)

Load - in case re-plotting needed

In [ ]:
cnf_matrix = metrics.confusion_matrix(test_predicted, y_test_labels)
print(cnf_matrix)

# Plot confusion matrix
plt.figure(figsize=(25,25)) 
plot_confusion_matrix(cnf_matrix, classes=class_label)

Predict class for test set images

Calculate accuracy using manual calculation

In [ ]:
num_images = len(correct)
print("Accuracy: %.2f%%" % ((sum(correct)*100)/num_images))

Displaying miscalssified image sets (and display the unqiue labels of misclassified fruits)

In [ ]:
images_error, images_error_label = visualize_errors(test_fruit_img, y_test_labels, class_label, test_predicted, correct)

## Random display some misclassified images

In [ ]:
# Total number of misclassified images
len(images_error)

In [ ]:
#images_error

display = 10 # number of images to display

plt.figure(figsize=(12,12))
idx = np.random.choice(len(images_error), display ,replace=False)

image = images_error[idx]
image = np.transpose(image, (0, 3, 1, 2))
image = image.astype(np.float32, copy=False)

image -= mean
image /= std

p = network.predict(image)
for i in range(len(idx)):
    plt.subplot(display,2,2*i+1)
    plt.tight_layout()
    plt.imshow(images_error[idx[i]])
    plt.title(id_to_label[images_error_label[idx[i]]])

    pred_label = np.argsort(-p[i])[:5]
    pred_prob = [p[i][l] for l in pred_label]
    pred_label = [id_to_label[l] for l in pred_label]
    
    plt.subplot(display,2,2*i+2)
    plt.tight_layout()
    plt.bar(range(5),pred_prob)
    plt.xticks(range(5), pred_label)

plt.show()

# Main evaluation and code ends here

## Additional support codes - Predict an image with the trained model

Load an image

In [ ]:
image = cv2.imread('Validation_down/Apple Golden 2/3_100.jpg',cv2.IMREAD_COLOR)
image = cv2.resize(image, (32, 32))
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

In [ ]:
plt.imshow(image, cmap='gray')  

Perform the same image preprocess

In [ ]:
image = np.array([image]) # to add one more axis

image = np.transpose(image, (0, 3, 1, 2))
image = image.astype(np.float32, copy=False)

image -= mean
image /= std

Predict the image

In [ ]:
predicted = network.predict(image)
predicted

In [ ]:
# Return the top 5 classes
index = np.argsort(-predicted, axis=1)[0][:5] #negative for descending
index

Display the top 5 classes with their probabilities

In [ ]:
for i in index:
  print('Class {}:\t{}\t\t\tProbability: {}'.format(i, id_to_label[i], predicted[0][i]))